# Henter data
Henter inn data om luftkvalitet fra Blindern i Oslo i løpet av hele 2017. 


In [1]:
import requests
import pandas as pd
import json
import sys

# API URL
url = "https://api.nilu.no/aq/historical/2017-01-01/2017-12-31/59.89869/10.81495/3.0?components=no2;pm10"

#sender et get-kall til API-endepunketet og lagrer svaret response. 
response = requests.get(url)

def get_data():
    #Dersom statuskoden er 200 vil dataen lagres.
    if response.status_code == 200:
        
        data = response.json()  

        with open("../data/luftkvalitet_2017.json","w") as f:
            json.dump(data, f, indent=4)
        print("Dataen er lagret i luftkvalitet_2017.json")
    #Dersom statuskoden er noe annet enn 200 vil vi få beskjed om at noe gikk galt. 
    else:
        print("noe gikk galt med API-kallet. Feilkoden er:", response.status_code)

get_data()












Dataen er lagret i luftkvalitet_2017.json


# Oppretter kolloner for målingene av NO2 og PM10
Vil endre formatet til values. Gjør derfor om målingene i values til en tabell og legger til kolonnene "PM10" og "NO2". Legger så til kolonnen "PM10" over til dataframen "df_NO" slik at vi har målingene i samme dataframe. Endrer så navn på dataframen til "maalinger_2017". 

In [2]:
import pandas as pd
import json

sys.path.append('../src')

from Weather_analysis import DataCleaner 

with open('luftkvalitet_2017.json', 'r') as file:
    data = json.load(file)

#Bruker funskjonen extract_component 
extractor = DataCleaner(data)  
df_NO2 = extractor.extract_component_data("NO2")
df_PM10 = extractor.extract_component_data("PM10")

#Endrer navn på de nye kolonnene
df_NO2 = extractor.rename_columns(df_NO2, {'value': 'NO2'})
df_PM10 = extractor.rename_columns(df_PM10, {'value': 'PM10'})

#Legger til kolonnene i same dataframe
df_NO2['PM10'] = df_PM10['PM10']

#Endrer navn
maalinger_2017=df_NO2


maalinger_2017







FileNotFoundError: [Errno 2] No such file or directory: 'luftkvalitet_2017.json'

# Fjerner unødvendige kolonner
Fjerner kolonnene qualityControlled, index, color, component for å gjøre tabellen mer oversiktelig. Legger også til kolonnen "unit" som oppgir enheten til PM10- og NO2-målingene. I tilegg runder jeg av måleverdiene til maks 3 desimaler. 

In [ ]:

#Fjerner kolonner
cleaner = DataCleaner(maalinger_2017)
kolonner_som_skal_bort = ['qualityControlled', 'index', 'color', 'component']
maalinger_2017 = cleaner.drop_columns(kolonner_som_skal_bort)

#legger til kolonnen unit
maalinger_2017['unit']='µg/m³'
#endrer til maks tre desimaler
maalinger_2017[['NO2', 'PM10']] = maalinger_2017[['NO2', 'PM10']].round(3)



maalinger_2017


,fromTime,toTime,NO2,PM10,unit
0,2017-01-01T00:00:00+01:00,2017-01-01T01:00:00+01:00,33.656,305.36,µg/m³
1,2017-01-01T01:00:00+01:00,2017-01-01T02:00:00+01:00,66.924,258.06,µg/m³
2,2017-01-01T02:00:00+01:00,2017-01-01T03:00:00+01:00,45.079,103.84,µg/m³
3,2017-01-01T03:00:00+01:00,2017-01-01T04:00:00+01:00,63.750,54.23,µg/m³
4,2017-01-01T04:00:00+01:00,2017-01-01T05:00:00+01:00,60.041,35.75,µg/m³
...,...,...,...,...,...
8689,2017-12-30T19:00:00+01:00,2017-12-30T20:00:00+01:00,51.414,12.43,µg/m³
8690,2017-12-30T20:00:00+01:00,2017-12-30T21:00:00+01:00,27.270,14.30,µg/m³
8691,2017-12-30T21:00:00+01:00,2017-12-30T22:00:00+01:00,20.334,12.65,µg/m³
8692,2017-12-30T22:00:00+01:00,2017-12-30T23:00:00+01:00,23.900,11.55,µg/m³


# Endrer på tidssoppsettet
 Endrer på tidsoppsettet ettersom tidsoppsetett med 'fromTime' og 'toTime' ikker var oversiktelig. Konverterer verdiene i 'fromTime' kolonnen fra tekststrenger til datetime-objekter slik at det blir lettere å jobbe med.  Lager kolonnene 'Date' som oppgir dato og 'time' som oppgir klokkeslett for målingene. Etter at de nye kolonnene er opprettet sletter vi kolonnene 'fromTime' og 'toTime'. Vi velger så å bytte plass på kolonne slik at vi får de i rekkefølgen 'Date', 'Time', 'NO2', 'PM10','unit'. 
 

In [ ]:
#Konverterer verdiene i 'fromTime' kolonnen fra tekststrenger til datetime-objekter.
maalinger_2017['fromTime'] = pd.to_datetime(maalinger_2017['fromTime'])

#Oppretter en ny kolonne 'Date' som viser datoen til målingene. 
maalinger_2017['Date'] = maalinger_2017['fromTime'].dt.date

#Oppretter en ny kolonne 'Time' som viser klokkeslettet for målingene. 
maalinger_2017['Time'] = maalinger_2017['fromTime'].dt.strftime('%H:%M')

#Sletter kolonnene 'fromTime' og 'toTime'.
cleaner = DataCleaner(maalinger_2017)
kolonner_som_skal_bort = ['fromTime','toTime']
maalinger_2017 = cleaner.drop_columns(kolonner_som_skal_bort)

#Endrer rekkefølge på kolonnene
maalinger_2017 = maalinger_2017[['Date', 'Time', 'NO2', 'PM10','unit']]


# Sjekker for manglende verdier og unormale verdier
Sjekker for manglende verdier, og finner ut at vi ikke mangler noe. I tilegg vil jeg sjekke om målingene av NO2 og PM10 gir mening. Jeg sjekker om alle målingene for hver time er under 400 µg/m. Verdier som er høyere enn 400 µg/m blir ikke lenger kategorisert av miljødirektoratets miljøklasser etterom de regnes som urealistiske. Men finner ut at alle målingene er innenfor gyldighetsområdet. 

In [ ]:

sys.path.append('../src')

from Weather_analysis import DataQualityChecker 

# Bruker funskjonen check_missing_values fra klassen DataQualityChecker.
# Denne funskjonen sjekker for manglende verdier. 
checker = DataQualityChecker(maalinger_2017)
missing = checker.check_missing_values()

#Bruker funksjonen find_extreme_values fra klassen DataQualityChecker. 
#Sjekker etter ekstreme verdier for NO2 og PM10
extreme_no2 = checker.find_extreme_values('NO2', 400)
extreme_pm10 = checker.find_extreme_values('PM10', 400)

maalinger_2017



Manglende verdier:
 Date    0
Time    0
NO2     0
PM10    0
unit    0
dtype: int64
Ekstreme verdier i NO2 over 400:
 Empty DataFrame
Columns: [Date, Time, NO2, PM10, unit]
Index: []
Ekstreme verdier i PM10 over 400:
 Empty DataFrame
Columns: [Date, Time, NO2, PM10, unit]
Index: []


,Date,Time,NO2,PM10,unit
0,2017-01-01,00:00,33.656,305.36,µg/m³
1,2017-01-01,01:00,66.924,258.06,µg/m³
2,2017-01-01,02:00,45.079,103.84,µg/m³
3,2017-01-01,03:00,63.750,54.23,µg/m³
4,2017-01-01,04:00,60.041,35.75,µg/m³
...,...,...,...,...,...
8689,2017-12-30,19:00,51.414,12.43,µg/m³
8690,2017-12-30,20:00,27.270,14.30,µg/m³
8691,2017-12-30,21:00,20.334,12.65,µg/m³
8692,2017-12-30,22:00,23.900,11.55,µg/m³


# Gjennomsnitt per dag
Dataen som er hentet inneholder målinger for hver time, derfor lager vi en ny oversikt med gjennomsnittet av målingene per døgn slik at den blir mer oversiktelig. Legger også til en kolonne som forteller oss enheten til målingene. Deretter velger jeg at målingene skal kun inneholde 3 desimaler. Til slutt konvertere jeg dataen daglig_gjennomsnitt til en csv-fil. Vi har altså nå ryddet og renset dataen slik at den er mye enklere å bruke videre. 


In [ ]:
# Grupper etter 'Date' og beregner gjennomsnittet for hver dag
daglig_gjennomsnitt = maalinger_2017.groupby('Date')[['NO2', 'PM10']].mean().reset_index()

#legger til en kolonne med enheter for verdiene
daglig_gjennomsnitt['Enhet'] = 'µg/m³'


# Runder av verdiene til 3 desimaler
daglig_gjennomsnitt[['NO2', 'PM10']] = daglig_gjennomsnitt[['NO2', 'PM10']].round(3)

#Koverterer dataen i "daglige_gjennomsnitt" til CSV_format. 
daglig_gjennomsnitt.to_csv('../data/daglig_gjennomsnitt_2017.csv')

